In [96]:
import pandas as pd
import numpy as np 
import datetime
inventory = pd.read_csv("data/inventory.csv")
products = pd.read_csv("data/products_ext.csv")
#note that this ^^^^ is the newly downlaoded dataset from canvas renamed to products_ext.csv
promotions = pd.read_csv("data/promotions.csv")
transactions = pd.read_csv("data/transactions.csv")

#create a reference dataframe
dayofyear = list(range(365))
date = [datetime.datetime(2018, 1, 1) + datetime.timedelta(day) for day in dayofyear]
df_date = pd.DataFrame({"day":dayofyear, "date":date})

import ast
for colname in inventory.columns[2:]:
    inventory[colname] = [ast.literal_eval(i) for i in inventory[colname]]

In [97]:
#specific products to be looking at
product_names = ['Biologische prei', 'Biologische brocolli', 'Biologische courgette', 'Biologische rode kool', 'Biologische cherry tomaten', 'Biologisch Bloemkool', 'Biologische pompoen', 'Biologische kikkererwten', 'Biologische knoflook']

In [98]:
transactions.head()

,day,time,customer,bank acount,category,product_id,description,size,std_sales_price,purchase_price,bio,basic
0,1/1/2018,9:07:00,0.0,NaN,snack,s_5,Eierkoeken,4st,1.79,1.79,0.0,0.0
1,1/1/2018,9:07:00,0.0,NaN,meat,m_16,Sappige gelders rookworst,250g,1.08,1.08,0.0,0.0
2,1/1/2018,9:07:00,0.0,NaN,meat,m_18,Varkensbraadworst,4st,2.49,1.99,NaN,0.0
3,1/1/2018,9:07:00,0.0,NaN,meat,m_6,Half om half gehakt,500g,2.69,2.42,0.0,0.0
4,1/1/2018,9:07:00,0.0,NaN,bread,b_33,Biologisch bruin heel,1st,2.12,2.12,1.0,0.0


In [99]:
inventory.head()

,day,before or after delivery,Komkommer,Courgette,SnoepVegetable tomaat,Paprika Mix,Paprika,Geschrapte worteltjes,Basis Sperziebonen,Basis Rode kool met appel,...,Roomboter chocolade brood,Chinois brioche,Vanille muffins (vers),Basic chocolate chip cookies,Oreo,Bakkersspeculaas,Eierkoeken,Ribbelchips naturel,Ribbelchips paprika,Doritos sweet chili
0,0,before,[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
1,0,after,"[(14, 36)]","[(14, 8)]","[(14, 37)]","[(14, 33)]","[(14, 34)]","[(14, 35)]","[(14, 31)]","[(14, 35)]",...,"[(5, 19)]","[(5, 23)]","[(100, 130)]","[(100, 45)]","[(100, 131)]","[(100, 45)]","[(100, 125)]","[(100, 45)]","[(100, 44)]","[(100, 128)]"
2,3,before,"[(14, 13)]",[],"[(14, 13)]","[(14, 11)]",[],[],"[(14, 2)]","[(14, 12)]",...,[],[],"[(100, 3)]",[],"[(100, 19)]","[(100, 7)]","[(100, 8)]","[(100, 10)]","[(100, 11)]","[(100, 26)]"
3,3,after,"[(14, 13), (17, 36)]","[(17, 8)]","[(14, 13), (17, 37)]","[(14, 11), (17, 33)]","[(17, 34)]","[(17, 35)]","[(14, 2), (17, 31)]","[(14, 12), (17, 35)]",...,"[(8, 19)]","[(8, 23)]","[(100, 3), (103, 130)]","[(103, 45)]","[(100, 19), (103, 131)]","[(100, 7), (103, 45)]","[(100, 8), (103, 125)]","[(100, 10), (103, 45)]","[(100, 11), (103, 44)]","[(100, 26), (103, 128)]"
4,7,before,"[(17, 35)]",[],"[(17, 27)]","[(17, 17)]","[(17, 1)]","[(17, 13)]","[(17, 8)]","[(17, 32)]",...,[],[],"[(103, 26)]","[(103, 19)]","[(103, 55)]","[(103, 16)]","[(103, 39)]","[(103, 28)]","[(103, 19)]","[(103, 58)]"


In [100]:
inventory_filt = inventory[['day', 'before or after delivery', 'Biologische prei', 'Biologische brocolli', 'Biologische courgette', 'Biologische rode kool', 'Biologische cherry tomaten', 'Biologische Bloemkool', 'Biologische pompoen', 'Biologische kikkererwten', 'Biologische knoflook']]
products_filt = products
transactions_filt = transactions[transactions['description'].isin(product_names)]

In [101]:
def day_of_year_converter(row):
    #converted_date = datetime.datetime(2018, 1, 1) + datetime.timedelta(row["day"])
    return pd.to_datetime(row['day']).dayofyear - 1

#inventory_filt["DOY"] = inventory_filt.apply(date_converter, axis = 1)

In [102]:
transactions_filt['DOY'] = transactions_filt.apply(day_of_year_converter, axis = 1)

<ipython-input-102-04a6aaab1626>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactions_filt['DOY'] = transactions_filt.apply(day_of_year_converter, axis = 1)


In [103]:
transactions_filt[transactions_filt['description'] == 'Biologische prei'].groupby('DOY').count()

,day,time,customer,bank acount,category,product_id,description,size,std_sales_price,purchase_price,bio,basic
DOY,,,,,,,,,,,,
0.0,2,2,2,1,2,2,2,2,2,2,2,2
1.0,1,1,1,0,1,1,1,1,1,1,1,1
2.0,3,3,3,3,3,3,3,3,3,3,3,3
3.0,9,9,9,9,9,9,9,9,9,9,9,9
4.0,4,4,4,4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...
359.0,4,4,4,4,4,4,4,4,4,4,4,4
360.0,4,4,4,4,4,4,4,4,4,4,4,4
361.0,2,2,2,1,2,2,2,2,2,2,2,2


In [104]:
#loss by supermarket (don't forget promotion weeks) purchase price/std price in transactions
#don't forget to download new data from canvas for purchase price
#now apply to whole dataset with for loop over all the products

In [105]:
# product_list = list(inventory.columns[2:])
product_list = list(inventory.columns[2:-2])
for element in ['Blauwe bessen', 'Blauwe bessen.1', 'Rundergehakt', 'Unox Gelderse rookworst', 'Rundergehakt.1', 'Unox Gelderse rookworst.1', 'Biologische spekreepjes', 'Sappige gelders rookworst', 'Biologisch rundergehakt', 'Biologisch rundergehakt.1', 'Spekblokjes', 'biologische Hamburger', 'Biologische gerookte ontbijtspek', 'Biologische runderchipolata', 'Biologische rundersoepballetjes']:
    product_list.remove(element)

In [106]:
#creation of product waste table:
def date_converter(row):
    converted_date = datetime.datetime(2018, 1, 1) + datetime.timedelta(row["day"])
    return converted_date

inventory["date"] = inventory.apply(date_converter, axis = 1) #convert day of year to actual timestamp
def week_of_year(row):
    week_nr = datetime.date(row["date"].year, row["date"].month, row["date"].day).isocalendar()[1]
    return week_nr

inventory["week"] = inventory.apply(week_of_year, axis = 1)
def check_discount(row):
    if row["week"] in promotions_filter["week"].values:
        discount_amount = promotions_filter[promotions_filter["week"] == row["week"]]["discount"]
        return discount_amount.values[0]
    else:
        return 0

def total_inv(row):
    total = 0
    if len(row[product]) > 0:
        for batch in row[product]:
            total += batch[1]
    return total
#adds total nr items in inventory for the filtered dataframe

def input_inv(row):
    for element_list in filter_inventory[product]:
        if len(element_list) > 0:
            for batch in element_list:
                if batch[0] == row["best before"]:
                    return batch[1]

transactions["day"] = pd.to_datetime(transactions["day"])
first_restock = df_date[df_date["day"] == 3]["date"].values[0]

waste_metrics_df = pd.DataFrame(columns = ['product', 'waste', 'lost_Revenue', 'lost_profit', 'cost_wasted'])

for product in product_list:
    filter_inventory = inventory[["day", "before or after delivery", "date", "week", product]]
    promotions_filter = promotions[promotions["description"] == product]
    filter_inventory["total inventory"] = filter_inventory.apply(total_inv, axis = 1)
    filter_transactions = transactions[transactions["description"] == product]

    best_before = []
    for element_list in filter_inventory[product]:
        for batch in element_list:
            best_before.append(batch[0])
    bb_dates = np.unique(best_before)

    df_waste = pd.DataFrame({"best before":bb_dates})
    #df_waste #build initial df with points where waste can occur
    df_waste["amount"] = df_waste.apply(input_inv, axis = 1)
    df_waste = df_waste[df_waste["best before"] < 365] #only keep rows where expiry date is in same year

    #first_restock check purchases mde before the first restock
    filter_transactions[filter_transactions["day"] < first_restock]

    #cumulative sum of purchases from that product
    purchases_per_day = filter_transactions.groupby("day").count()["product_id"].cumsum()
    #purchases made per day from that product
    purchases_per_day_plotting = filter_transactions.groupby("day").count()["product_id"]
    #idk know what is this
    purchases_per_day_plotting.index = purchases_per_day_plotting.index.dayofyear/7

    purchases_per_day.index[0].year
    weeknr = []
    for ind in purchases_per_day.index:
        week = datetime.date(ind.year, ind.month, ind.day).isocalendar()[1]
        weeknr.append(week)
    #week number for each purchase day


    df_waste.drop(index=df_waste.tail(9).index, inplace=True)
    #drop the last rows not to cause index issues
    #STILL NEED TO BE FIXED

    df_waste["purchases"] = [purchases_per_day[i-1] for i in df_waste["best before"]]

    df_waste["prev purchases"] = df_waste["purchases"].diff()

    length = df_waste.shape[0]
    lst_waste = [] #nr rows in waste dataframe
    #amount left from the item
    df_waste["remaining"] = df_waste["amount"] - df_waste["prev purchases"] 
    #the first row is different as there is no prev pruchases available
    df_waste["remaining"].iloc[0] = df_waste["amount"].iloc[0] - df_waste["purchases"].iloc[0]

    for i in range(length):
        #iterates through the rows of the waste dataframe
        current_row = df_waste.iloc[i]
        if i == 0:
            #
            lst_waste.append(current_row["remaining"])
            continue     
        else:
            prev_row = df_waste.iloc[i-1] #previous row in dataframe
            if lst_waste[-1] < 0:
                #no idea whats happening here, I've had enough
                waste = lst_waste[-1] + current_row["remaining"]
                lst_waste.append(lst_waste[-1] + current_row["remaining"])
            else:
                lst_waste.append(current_row["remaining"])
        
    df_waste["waste"] = lst_waste
    def date_converter(row):
        converted_date = datetime.datetime(2018, 1, 1) + datetime.timedelta(row["best before"])
        return converted_date
    purchase_price = products[products['description'] == product]['purchase_price']
    df_waste["date"] = df_waste.apply(date_converter, axis = 1)
    df_waste["week"] = df_waste.apply(week_of_year, axis = 1)
    df_waste[df_waste["waste"] < 0]["waste"] = 0
    df_waste['weekPrice'] = [float(products[products['description'] == product]['std_sales_price'])] * len(df_waste)
    #df_waste.set_index('week', inplace = True)
    #print(df_waste.head())
    for week_ind in list(promotions[promotions['description'] == product].index):
        week = promotions.at[week_ind, 'week']
        #print(week_ind)
        # print(df_waste[['week']].dtypes)
        if week in list(np.unique(df_waste.week)):
            #print(week)
            df_waste.loc[df_waste[df_waste['week'] == week]['weekPrice'].index, 'weekPrice'] = (1 - (promotions.at[week_ind, 'discount'] / 100)) * df_waste.loc[df_waste[df_waste['week'] == week]['weekPrice'].index, 'weekPrice']
            #break
    df_waste['lost_Revenue'] = df_waste.waste * df_waste.weekPrice
    df_waste['purc_price'] = [float(purchase_price)] * len(df_waste)
    df_waste['lost_profit'] = df_waste['waste'] * (df_waste['weekPrice'] - df_waste['purc_price'])
    df_waste['cost_wasted'] = df_waste['waste'] * df_waste['purc_price']

    metrics = list(df_waste[df_waste['waste']>0][['waste', 'lost_Revenue', 'lost_profit', 'cost_wasted']].sum())

    new_row = [product] + metrics

    waste_metrics_df.loc[len(waste_metrics_df)] = new_row
    print('{} is Complete'.format(product))

#product = 'Biologische knoflook' #select the product to analyze

<ipython-input-106-246ea44bbd80>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_inventory["total inventory"] = filter_inventory.apply(total_inv, axis = 1)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-106-246ea44bbd80>:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [88]:
waste_metrics_df.sort_values(by='lost_Revenue', ascending=False, inplace = True)
waste_metrics_df.reset_index(inplace=True, drop=True)
waste_metrics_df.head(30)

,index,product,waste,lost_Revenue,lost_profit,cost_wasted
0,116,Biologisch Vloerbrood meerzaden heel,2065.0,5625.0390,3952.3890,1672.65
1,122,Biologisch bruin heel,1796.0,3801.3720,2651.9320,1149.44
2,121,Biologisch volkoren heel,1659.0,3751.2540,2473.8240,1277.43
3,61,Biologische sinaasappelen,816.0,2790.2550,1900.8150,889.44
4,14,Biologische pompoen,1040.0,2561.6250,1948.0250,613.60
5,12,Biologische Bloemkool,895.0,2097.8225,1605.5725,492.25
6,11,Biologische courgette,1187.0,1992.8480,1482.4380,510.41
7,19,Biologische brocolli,1110.0,1969.9845,1514.8845,455.10
8,59,Biologische santana appelen,674.0,1767.4645,1228.2645,539.20
9,56,Basis annanasstukjes,1828.0,1741.4925,1193.0925,548.40


In [79]:
waste_metrics_df.sum()

index                                                        9591
product         Biologisch Vloerbrood meerzaden heelBiologisch...
waste                                                       50972
lost_Revenue                                              85054.6
lost_profit                                               61089.6
cost_wasted                                               23964.9
dtype: object

In [86]:
waste_metrics_df[waste_metrics_df['product'].isin(product_names)].sum()

index                                                         123
product         Biologische courgetteBiologische rode koolBiol...
waste                                                        8305
lost_Revenue                                              11803.2
lost_profit                                               8998.41
cost_wasted                                               2804.78
dtype: object

In [93]:
waste_metrics_df.head(13)['product'].tolist()

['Biologisch Vloerbrood meerzaden heel',
 'Biologisch bruin heel',
 'Biologisch volkoren heel',
 'Biologische sinaasappelen',
 'Biologische pompoen',
 'Biologische Bloemkool',
 'Biologische courgette',
 'Biologische brocolli',
 'Biologische santana appelen',
 'Basis annanasstukjes',
 'Biologische cherry tomaten',
 'Biologische rozijnen',
 'Biologische rode kool']